In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/worachotn/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/worachotn/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/worachotn/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/worachotn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
from nltk.corpus import wordnet
from nltk.corpus import stopwords

lemmatizer = nltk.stem.WordNetLemmatizer()  # Initiate nltk lemmatizer

In [3]:
import math
import os
import pprint
import logging

import datasets
import nltk
import numpy as np
import torch
from tqdm.auto import tqdm

import transformers
from accelerate import Accelerator
from filelock import FileLock
from transformers import AdamW, get_scheduler, set_seed

from transformers.file_utils import is_offline_mode
from transformers.utils.versions import require_version

from args import parse_args
from data_loader import raw_data_loader, data_processor
from model_loader import model_loader
from rouge_s import py_rouge_scores
from utils import label_smoothed_nll_loss, postprocess_text

[nltk_data] Downloading package punkt to /home/worachotn/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/worachotn/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/worachotn/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/worachotn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)

In [5]:
from transformers import (
    MODEL_MAPPING,
    SchedulerType,
)

# You should update this to your particular problem to have better documentation of `model_type`
MODEL_CONFIG_CLASSES = list(MODEL_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

In [6]:
import argparse
arg_parser = argparse.ArgumentParser(description="BART")
arg_parser.add_argument("--len_input", dest="len_input", type=str, default=None, help="set up prefix input",choices=('no', 'topic', 'length', 'topic-length', 'length-topic', 'simple', 'simple-topic-tagger', 'simple-tagger'))
arg_parser.add_argument("--len_output", dest="len_output", default=None, help="Use the ctrlen model or not", choices=('no', 'topic', 'length', 'topic-length', 'length-topic'))
arg_parser.add_argument("--output_dir", dest="output_dir", type=str, default="./output/1", help="default")
arg_parser.add_argument("--train_file", dest="train_file", type=str, default=None, help="A csv or a json file containing the training data.")
arg_parser.add_argument("--validation_file", dest="validation_file", type=str, default=None, help="A csv or a json file containing the validation data.")
arg_parser.add_argument("--test_file", dest="test_file", type=str, default=None, help="A csv or a json file containing the test data.")
arg_parser.add_argument("--ignore_pad_token_for_loss", dest="ignore_pad_token_for_loss", type=bool, default=True, help="Whether to ignore the tokens corresponding to " "padded labels in the loss computation or not.",)
arg_parser.add_argument("--text_column", dest="text_column", type=str, default="dialogue", help="The name of the column in the datasets containing the full texts (for summarization).")
arg_parser.add_argument("--summary_column", dest="summary_column", type=str, default="summary", help="The name of the column in the datasets containing the summaries (for summarization).")
arg_parser.add_argument("--model_name_or_path", dest="model_name_or_path", type=str, default="facebook/bart-large", help="Path to pretrained model or model identifier from huggingface.co/models.")
arg_parser.add_argument("--model_type", dest="model_type", type=str, default="bart", help="Model type to use if training from scratch.", choices=MODEL_TYPES)
arg_parser.add_argument("--max_source_length", dest="max_source_length", type=int, default=1024, help="default")
arg_parser.add_argument("--source_prefix", dest="source_prefix", type=str, default=None, help="A prefix to add before every source text " "(useful for T5 models).")
arg_parser.add_argument("--preprocessing_num_workers", type=int, default=None, help="The number of processes to use for the preprocessing.")
# arg_parser.add_argument("--overwrite_cache", dest="overwrite_cache", type=lambda x:bool(strtobool(x)), default=True, help="default")
arg_parser.add_argument("--overwrite_cache", dest="overwrite_cache", type=bool, default=None, help="Overwrite the cached training and evaluation sets")
arg_parser.add_argument("--min_target_length", dest="min_target_length", type=int, default=1, help="The minimal total sequence length for target text")
arg_parser.add_argument("--max_target_length", dest="max_target_length", type=int, default=128, help="The maximum total sequence length for target text after "
        "tokenization. Sequences longer than this will be truncated, sequences shorter will be padded."
        "during ``evaluate`` and ``predict``.")
arg_parser.add_argument("--num_beams", dest="num_beams", type=int, default=4, help="Number of beams to use for evaluation. This argument will be "
        "passed to ``model.generate``, which is used during ``evaluate`` and ``predict``.")
arg_parser.add_argument("--learning_rate", dest="learning_rate", type=float, default=5e-5, help="Initial learning rate (after the potential warmup period) to use.")
arg_parser.add_argument("--pad_to_max_length", action="store_true", help="If passed, pad all samples to `max_length`. Otherwise, dynamic padding is used.",)
arg_parser.add_argument("--weight_decay", dest="weight_decay", type=float, default=1e-3, help="Weight decay to use.")
arg_parser.add_argument("--label_smoothing", dest="label_smoothing", type=float, default=0.1, help="hyperparameter for label smoothing.")
arg_parser.add_argument("--length_penalty", dest="length_penalty", type=float, default=1.0, help="large - longer sequence, small - shorter sequence")
arg_parser.add_argument("--num_train_epochs", dest="num_train_epochs", type=int, default=15, help="Total number of training epochs to perform.")
arg_parser.add_argument("--per_device_train_batch_size", dest="per_device_train_batch_size", type=int, default=8, help="Batch size (per device) for the training dataloader.")
arg_parser.add_argument("--gradient_accumulation_steps", dest="gradient_accumulation_steps", type=int, default=64, help="Number of updates steps to accumulate before performing a backward/update pass.")
arg_parser.add_argument("--per_device_eval_batch_size", dest="per_device_eval_batch_size", type=int, default=8, help="Batch size (per device) for the evaluation dataloader.")
arg_parser.add_argument("--per_device_test_batch_size", dest="per_device_test_batch_size", type=int, default=8, help="Batch size (per device) for the evaluation dataloader.")
arg_parser.add_argument("--num_warmup_steps", dest="num_warmup_steps", type=int, default=0, help="Number of steps for the warmup in the lr scheduler.")
arg_parser.add_argument("--cache_dir", dest="cache_dir", type=str, default="./output/cache", help="default")
arg_parser.add_argument("--seed", dest="seed", type=int, default=12345, help="default")
# arg_parser.add_argument("-f", required=False) #important
arg_parser.add_argument("--config_name", type=str, default=None, help="Pretrained config name or path if not the same as model_name")
arg_parser.add_argument("--tokenizer_name", type=str, default=None, help="Pretrained tokenizer name or path if not the same as model_name")
arg_parser.add_argument("--use_slow_tokenizer", dest="use_slow_tokenizer", action="store_true", help="If passed, will use a slow tokenizer (not backed by the 🤗 Tokenizers library).")
arg_parser.add_argument("--max_train_steps", type=int, default=None, help="Total number of training steps to perform. If provided, overrides num_train_epochs.")
arg_parser.add_argument("--lr_scheduler_type", type=SchedulerType, default="linear", help="The scheduler type to use.", choices=["linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup"])
arg_parser.add_argument("--ctrlen_model", action='store_true', default=False, help="Use the ctrlen model or not")
arg_parser.add_argument("--sim_window_size", type=int, default=5, help="window size for computing loss.")
arg_parser.add_argument("--sim_loss", type=float, default=0, help="the loss weight for similarity scores.")
arg_parser.add_argument("--special_len_token_init", type=str, default=None, help="ways to initialize special token for length (random, zero, token_embs)")
arg_parser.add_argument("--embedding_lr", type=float, default=5e-5, help="Initial learning rate for embedding layers.")
arg_parser.add_argument("--len_start", type=int, default=1, help="start length.")
arg_parser.add_argument("--len_end", type=int, default=100, help="end length.")
arg_parser.add_argument("--data_aug",action='store_true',default=False,help="whether to perform data augmentation or not")
arg_parser.add_argument("--pred_len", action='store_true', default=False, help="whether to use the golden length or predicted length")
arg_parser.add_argument("--shuffle", action='store_true', default=False, help="whether to shuffle the dataset to balance train/validation/test")
arg_parser.add_argument("--debug", action='store_true', default=False, help="Use the debug mode or not")

arg_parser.add_argument("--topic_tagger", dest="topic_tagger", type=bool, default=None, help="Use topic tag <TAG> or not")
arg_parser.add_argument("--contrastive_loss", dest="contrastive_loss", type=bool, default=False, help="Use contrastive loss or not")
arg_parser.add_argument("--alpha", dest="alpha", type=float, default=0.5, help="ration of computation loss in encoder_hidden_layer")

_StoreAction(option_strings=['--alpha'], dest='alpha', nargs=None, const=None, default=0.5, type=<class 'float'>, choices=None, required=False, help='ration of computation loss in encoder_hidden_layer', metavar=None)

In [7]:
args = arg_parser.parse_args('')

In [8]:
args.train_file = "./data/dialogtest/dialogsum.train.jsonl"
args.validation_file = "./data/dialogtest/dialogsum.dev.jsonl"
args.test_file = "./data/dialogtest/dialogsum.test.jsonl"
args.text_column = "dialogue"
args.summary_column = "summary"
args.model_name_or_path = "facebook/bart-large"
args.model_type = "bart"
args.max_source_length = 1024
args.min_target_length = 1
args.max_target_length = 128
args.num_beams = 4
args.learning_rate = 5e-5
args.weight_decay = 1e-3
args.label_smoothing = 0.1
args.length_penalty = 1.0
args.num_train_epochs = 5
args.per_device_train_batch_size = 2
args.gradient_accumulation_steps = 64
args.per_device_eval_batch_size = 8
args.per_device_test_batch_size = 8
args.num_warmup_steps = 0
args.cache_dir = "./output/cache"
args.overwrite_cache = True
args.seed = 12345

args.len_input = 'topic-length'
args.len_output = 'no'
args.output_dir = "./output/1-bart-baseline-loss"

args.topic_tagger = True
args.contrastive_loss = True

In [9]:
print(args.len_input)
print(args.topic_tagger)

topic-length
True


In [10]:
raw_datasets = raw_data_loader(args)

In [11]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'negative_dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'negative_dialogue', 'summary', 'topic'],
        num_rows: 50
    })
    test: Dataset({
        features: ['id', 'dialogue', 'negative_dialogue', 'summary', 'topic'],
        num_rows: 150
    })
})

In [12]:
raw_datasets['train']['negative_dialogue'][0]

"Topic of Summary: <TAG>discuss stress</TAG>. Length of Summary: 30. Dialogue: # Person1 # : Hi , Mr. Smith . I 'm Doctor Hawkins . Why are you here today ? # Person2 # : I found it would be a good idea to <TAG>get</TAG> a <TAG>check-up</TAG> . # Person1 # : Yes , well , you have n't had one for 5 years . You should have one every year . # Person2 # : I know . I figure as long as there is nothing wrong , why go see the doctor ? # Person1 # : Well , the best way to avoid serious illnesses is to find out about them early . So try to come at least once a year for your own good . # Person2 # : Ok. # Person1 # : Let me see here . Your eyes and ears look fine . Take a deep breath , please . Do you smoke , Mr. Smith ? # Person2 # : Yes . # Person1 # : Smoking is the leading cause of lung cancer and heart disease , you know . You really should quit . # Person2 # : I 've tried hundreds of times , but I just ca n't seem to kick the habit . # Person1 # : Well , we have classes and some medication

In [13]:
accelerator = Accelerator(mixed_precision="fp16")
logger.info(accelerator.state)

10/22/2023 13:06:12 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp16



In [14]:
logger.setLevel(logging.INFO if accelerator.is_local_main_process else logging.ERROR)
accelerator.is_local_main_process
datasets.utils.logging.set_verbosity_warning()
transformers.utils.logging.set_verbosity_info()
set_seed(args.seed)
torch.backends.cudnn.enabled = False
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
accelerator.is_main_process
os.makedirs(args.output_dir, exist_ok=True)
accelerator.wait_for_everyone()

In [15]:
config, tokenizer, model = model_loader(accelerator, logger, args)

loading configuration file config.json from cache at ./output/cache/models--facebook--bart-large/snapshots/cb48c1365bd826bd521f650dc2e0940aee54720c/config.json
Model config BartConfig {
  "_name_or_path": "facebook/bart-large",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2

In [19]:
print(dir(model))

['T_destination', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_auto_class', '_backward_compatibility_gradient_checkpointing', '_backward_hooks', '_backward_pre_hooks', '_buffers', '_call_impl', '_check_and_enable_flash_attn_2', '_convert_head_mask_to_5d', '_copy_lm_head_original_to_resized', '_create_repo', '_dispatch_accelerate_model', '_expand_inputs_for_generation', '_extract_past_from_model_output', '_forward_hooks', '_forward_hooks_with_kwargs', '_forward_pre_hooks', '_forward_pre_hooks_with_kwargs', '_from_config', '_get_backward_hooks', '_get_backward_pre_hooks', '_get_decoder_start_token_id', '_get_files_timesta

In [17]:
model.vocab_size

50267

In [23]:
print(dir(tokenizer))

['SPECIAL_TOKENS_ATTRIBUTES', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_add_tokens', '_additional_special_tokens', '_auto_class', '_batch_encode_plus', '_bos_token', '_call_one', '_cls_token', '_compile_jinja_template', '_convert_encoding', '_convert_id_to_token', '_convert_token_to_id_with_added_voc', '_create_repo', '_decode', '_decode_use_source_tokenizer', '_encode_plus', '_eos_token', '_eventual_warn_about_too_long_sequence', '_eventually_correct_t5_max_length', '_from_pretrained', '_get_files_timestamps', '_get_padding_truncation_strategies', '_in_target_context_manager', '_mask_token', '_pad', '_pad_token', '_pad_token_type_id', '_processo

In [27]:
tokenizer.SPECIAL_TOKENS_ATTRIBUTES

['bos_token',
 'eos_token',
 'unk_token',
 'sep_token',
 'pad_token',
 'cls_token',
 'mask_token',
 'additional_special_tokens']

In [18]:
print(tokenizer.additional_special_tokens)

['<TAG>', '</TAG>']


In [22]:
print(tokenizer.encode(raw_datasets['train']['negative_dialogue'][0]))

[0, 48931, 9, 19584, 35, 1437, 50265, 18909, 4781, 3992, 50266, 4, 41852, 9, 19584, 35, 389, 4, 33854, 35, 849, 18404, 134, 849, 4832, 12289, 2156, 427, 4, 1259, 479, 38, 128, 119, 12521, 15633, 479, 2612, 32, 47, 259, 452, 17487, 849, 18404, 176, 849, 4832, 38, 303, 24, 74, 28, 10, 205, 1114, 7, 1437, 50265, 6460, 50266, 10, 1437, 50265, 15954, 12, 658, 50266, 479, 849, 18404, 134, 849, 4832, 3216, 2156, 157, 2156, 47, 33, 295, 75, 56, 65, 13, 195, 107, 479, 370, 197, 33, 65, 358, 76, 479, 849, 18404, 176, 849, 4832, 38, 216, 479, 38, 1955, 25, 251, 25, 89, 16, 1085, 1593, 2156, 596, 213, 192, 5, 3299, 17487, 849, 18404, 134, 849, 4832, 2647, 2156, 5, 275, 169, 7, 1877, 1473, 14971, 16, 7, 465, 66, 59, 106, 419, 479, 407, 860, 7, 283, 23, 513, 683, 10, 76, 13, 110, 308, 205, 479, 849, 18404, 176, 849, 4832, 5148, 4, 849, 18404, 134, 849, 4832, 2780, 162, 192, 259, 479, 2486, 2473, 8, 12137, 356, 2051, 479, 4624, 10, 1844, 8016, 2156, 2540, 479, 1832, 47, 4603, 2156, 427, 4, 1259, 1748

In [280]:
class CustomDataCollatorForSeq2Seq(DataCollatorForSeq2Seq):
    def __init__(self, tokenizer, model, label_pad_token_id, pad_to_multiple_of, id_field="id"):
        super().__init__(tokenizer, model, label_pad_token_id, pad_to_multiple_of)
        self.id_field = id_field

    def collate_batch(self, features):
        batch = super().collate_batch(features)

        # Extract and add the "id" field to the batch
        ids = [feature[self.id_field] for feature in features]
        batch["id"] = id

        return batch

In [288]:
def data_processor(logger, args, accelerator, raw_datasets, tokenizer, model):
    ''' prepare dataset format for train/val/test '''
    def preprocess_function(examples):

        # summary - target
        targets = examples[summary_column]
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(targets, max_length=max_target_length, padding=padding, truncation=True)

        if args.ctrlen_model:
            gold_sum_len = [len(item) for item in labels['attention_mask']]

        # dialogue - input
        inputs = examples[text_column]
        new_inputs = []
        for i, inp in enumerate(inputs):
            if args.ctrlen_model:
                if 'pred_len' in examples:
                    new_inputs.append(prefix + "<len_{}> ".format(examples['pred_len'][i]) + inp)

                else:
                    new_inputs.append(prefix + "<len_{}> ".format(gold_sum_len[i]) + inp)
            else:
                new_inputs.append(prefix + inp)

        inputs = new_inputs
        id = examples['id']
        new_id = [x.split("_")[1] for x in id]
        # print(new_id)
        # print(type(new_id))
        # negative_dialogue = examples['negative_dialogue']
        model_inputs = tokenizer(inputs, max_length=args.max_source_length, padding=padding, truncation=True)
        # negative_inputs = tokenizer(inputs, max_length=args.max_source_length, padding=padding, truncation=True)
        # negative_inputs = tokenizer(negative_dialogue, max_length=args.max_source_length, padding=padding, truncation=True)
        # print(model_inputs)
        # print(len(model_inputs['input_ids'][0]))
        # print(len(negative_inputs['input_ids'][0]))
        # print(len(model_inputs['attention_mask']))

        # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
        # padding in the loss.
        if padding == "max_length" and args.ignore_pad_token_for_loss:
            labels["input_ids"] = [
                [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
            ]

        # model_inputs['neg_input_ids'] = negative_inputs['input_ids']
        # model_inputs['neg_attention_mask'] = negative_inputs['attention_mask']
        model_inputs["id"] = new_id
        # print(type(labels["input_ids"]))
        model_inputs["labels"] = labels["input_ids"]

        if args.ctrlen_model:
            model_inputs["gold_len"] = gold_sum_len

        return model_inputs

    prefix = args.source_prefix if args.source_prefix is not None else ""

    # Preprocessing the datasets.
    # First we tokenize all the texts.
    column_names = raw_datasets["train"].column_names

    # Get the column names for input/target.
    text_column = args.text_column
    if text_column not in column_names:
        raise ValueError(
            f"--text_column' value '{args.text_column}' needs to be one of: {', '.join(column_names)}"
        )

    summary_column = args.summary_column
    if summary_column not in column_names:
        raise ValueError(
            f"--summary_column' value '{args.summary_column}' needs to be one of: {', '.join(column_names)}"
        )

    # Temporarily set max_target_length for training.
    max_target_length = args.max_target_length
    padding = "max_length" if args.pad_to_max_length else False



    with accelerator.main_process_first():
        processed_datasets = raw_datasets.map(
            preprocess_function,
            batched=True,
            batch_size=1000,
            remove_columns=column_names,
            load_from_cache_file=not args.overwrite_cache,
            desc="Running tokenizer on dataset",
        )

    train_dataset = processed_datasets["train"]
    eval_dataset  = processed_datasets["validation"]
    test_dataset  = processed_datasets["test"]

    # Log a few random samples from the training set:
    for index in random.sample(range(len(train_dataset)), 1):
        logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")

    label_pad_token_id = -100 if args.ignore_pad_token_for_loss else tokenizer.pad_token_id
    # data_collator = DataCollatorForSeq2Seq(
    #     tokenizer,
    #     model=model,
    #     label_pad_token_id=label_pad_token_id,
    #     pad_to_multiple_of=8 if accelerator.use_fp16 else None,
    # )

    data_collator = CustomDataCollatorForSeq2Seq(        
        tokenizer,
        model=model,
        label_pad_token_id=label_pad_token_id,
        pad_to_multiple_of=8 if accelerator.use_fp16 else None,
        # padding='do_not_pad'
    )

    train_dataloader = DataLoader(train_dataset, shuffle=True, collate_fn=data_collator, batch_size=args.per_device_train_batch_size)
    eval_dataloader = DataLoader(eval_dataset, collate_fn=data_collator, batch_size=args.per_device_eval_batch_size)
    test_dataloader = DataLoader(test_dataset, collate_fn=data_collator, batch_size=args.per_device_test_batch_size)

    return (train_dataloader, eval_dataloader, test_dataloader), (train_dataset, eval_dataset, test_dataset)

In [289]:
dataloader, processed_dataset = data_processor(logger, args, accelerator, raw_datasets, tokenizer, model)
train_dataloader, eval_dataloader, test_dataloader = dataloader
train_dataset, _, _ = processed_dataset

Running tokenizer on dataset:   0%|          | 0/12460 [00:00<?, ? examples/s]

/home/worachotn/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3864: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/worachotn/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3864: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/worachotn/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3864: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of 

Running tokenizer on dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/1500 [00:00<?, ? examples/s]

/home/worachotn/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3864: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
10/14/2023 06:47:05 - INFO - __main__ - Sample 2649 of the training set: {'id': '2649', 'input_ids': [0, 48931, 9, 19584, 35, 81, 18288, 3588, 4, 41852, 9, 19584, 35, 883, 4, 33854, 35, 849, 41761, 134, 10431, 35, 407, 99, 109, 47, 206, 9, 127, 92, 3588, 116, 50118, 10431, 41761, 176, 10431, 35, 38, 206, 24, 18, 372, 328, 38, 269, 101, 5, 3195, 6, 8, 24, 1411, 269, 157, 19, 110, 2473, 4, 38, 348, 888, 57, 21754, 14, 3588, 23, 4471, 833, 18, 13, 10, 150, 122, 6, 53, 38, 938, 75, 686, 38, 115, 4960, 33035, 12, 9579, 1932, 13, 10, 3588, 23, 5, 1151, 4, 50118, 10431, 41761, 134, 10431,

8


/home/worachotn/.local/lib/python3.10/site-packages/accelerate/accelerator.py:523: FutureWarning: The `use_fp16` property is deprecated and will be removed in version 1.0 of Accelerate use `Accelerator.mixed_precision == 'fp16'` instead.
  warnings.warn(


In [284]:
train_dataset

Dataset({
    features: ['id', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 12460
})

In [285]:
train_dataset['id'][:5]

['0', '1', '2', '3', '4']

In [286]:
train_dataloader

In [287]:
for step, batch in enumerate(train_dataloader):
  # print(tokenizer.decode(batch['input_ids'][0]))
  print(batch)
  break

ValueError: -100 is not a valid PaddingStrategy, please select one of ['longest', 'max_length', 'do_not_pad']

In [134]:

# importing the required libraries 
import torch 
from torch.utils.data import Dataset 
from torch.utils.data import DataLoader 
  
# defining the Dataset class 
class data_set(Dataset): 
    def __init__(self): 
        numbers = list(range(0, 100, 1)) 
        self.data = numbers 
  
    def __len__(self): 
        return len(self.data) 
  
    def __getitem__(self, index): 
        return self.data[index] 
  
  
dataset = data_set() 
dataset2 = data_set() 
  
# implementing dataloader on the dataset and printing per batch 
dataloader = DataLoader(dataset, batch_size=10, shuffle=True) 
dataloader2 = DataLoader(dataset2, batch_size=10, shuffle=True) 
for i, batch in enumerate(dataloader): 
    print(i, batch) 
print("-"*100)
for i, batch in enumerate(dataloader2): 
    print(i, batch) 

0 tensor([33, 88, 51, 11, 82, 84, 32, 45, 52, 60])
1 tensor([54, 36, 26, 34, 65, 21,  2, 29, 38, 99])
2 tensor([59, 48, 25, 83, 17, 44, 96, 79, 49,  4])
3 tensor([23, 19, 50, 42, 16, 80, 89, 24, 27, 63])
4 tensor([93, 39,  0, 92, 64,  6, 94, 87, 18, 22])
5 tensor([43, 85, 41, 66, 67, 86, 77, 20, 97, 10])
6 tensor([71, 46, 72, 62, 91,  8, 15,  5, 98, 61])
7 tensor([78, 70, 68,  9, 74, 81, 90, 56, 75, 73])
8 tensor([30,  7,  1, 53, 37, 28,  3, 47, 57, 40])
9 tensor([95, 12, 69, 55, 13, 76, 31, 14, 58, 35])
----------------------------------------------------------------------------------------------------
0 tensor([ 4, 12, 27, 83, 42, 98, 87, 72, 51, 35])
1 tensor([46,  9, 85, 28, 26, 56, 88, 94, 31, 74])
2 tensor([49, 68, 44, 62, 54, 30, 79, 76, 90, 24])
3 tensor([ 2,  8,  6, 22, 91, 92, 73, 66, 50, 17])
4 tensor([95,  1, 57, 59,  5, 21, 53, 71, 16,  7])
5 tensor([93, 19, 23, 61, 25,  3, 47, 99, 77, 69])
6 tensor([63, 58, 38, 89, 81, 97, 67, 13, 41, 18])
7 tensor([34, 75,  0, 84, 43, 52

In [ ]:
"Unable to create tensor, you should probably activate truncation and/or padding with" 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your" features (`{key}` in this case) have excessive nesting (inputs type `list` where type `int` is" expected)."

In [ ]:
# = = = Training Preparation = = =
# Optimizer
# Split weights in two groups, one with weight decay and the other not.
no_decay = ["bias", "LayerNorm.weight"]

if args.ctrlen_model:
    no_decay_emb_matrix = ["bias", "LayerNorm.weight", "shared"]
else:
    no_decay_emb_matrix = ["bias", "LayerNorm.weight"]

optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay_emb_matrix)],
        "weight_decay": args.weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]

if args.ctrlen_model:
    if args.model_type == 'bart':
        optimizer_grouped_parameters.extend([{
            "params": model.seq2seq_model.model.shared.parameters(),
            "lr": args.embedding_lr}])
    elif args.model_type == 't5':
        optimizer_grouped_parameters.extend([{
            "params": model.seq2seq_model.shared.parameters(),
            "lr": args.embedding_lr}])
    else:
        raise ValueError('{} model type not implemented'.format(args.model_type))

# optimizer
optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
model, optimizer, train_dataloader, eval_dataloader, test_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader, test_dataloader
)

In [ ]:
# Scheduler and math around the number of training steps.
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / args.gradient_accumulation_steps)
if args.max_train_steps is None:
    args.max_train_steps = args.num_train_epochs * num_update_steps_per_epoch
else:
    args.num_train_epochs = math.ceil(args.max_train_steps / num_update_steps_per_epoch)

lr_scheduler = get_scheduler(
    name=args.lr_scheduler_type,
    optimizer=optimizer,
    num_warmup_steps=args.num_warmup_steps,
    num_training_steps=args.max_train_steps,
)

# = = = = = = = = = = = = = = = = Train = = = = = = = = = = = = = = = = = = =
total_batch_size = args.per_device_train_batch_size * accelerator.num_processes * args.gradient_accumulation_steps

logger.info("***** Running training *****")
logger.info(f" Num examples = {len(train_dataset)}")
logger.info(f" Num Epochs = {args.num_train_epochs}")
logger.info(f" Instantaneous batch size per device = {args.per_device_train_batch_size}")
logger.info(f" Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(f" Gradient Accumulation steps = {args.gradient_accumulation_steps}")
logger.info(f" Total optimization steps = {args.max_train_steps}")

# Only show the progress bar once on each machine.
progress_bar = tqdm(range(args.max_train_steps), desc="Training: ", disable=not accelerator.is_local_main_process)
completed_steps = 0

val_results = []
acc_losses  = []
best_r2_f1  = None
best_epoch  = 0

if args.model_type == 'bart' or args.model_type == 't5':
    task_specific_params = model.config.task_specific_params
    params = task_specific_params.get('summarization', {})
    params['min_length'] = args.min_target_length
    params['max_length'] = args.max_target_length
    params['length_penalty'] = args.length_penalty
    params['num_beams'] = args.num_beams
    model.config.update(params)
else:
    raise ValueError('{} model type not implemented'.format(args.model_type))

10/04/2023 18:57:09 - INFO - __main__ - ***** Running training *****
10/04/2023 18:57:09 - INFO - __main__ -  Num examples = 1500
10/04/2023 18:57:09 - INFO - __main__ -  Num Epochs = 5
10/04/2023 18:57:09 - INFO - __main__ -  Instantaneous batch size per device = 2
10/04/2023 18:57:09 - INFO - __main__ -  Total train batch size (w. parallel, distributed & accumulation) = 128
10/04/2023 18:57:09 - INFO - __main__ -  Gradient Accumulation steps = 64
10/04/2023 18:57:09 - INFO - __main__ -  Total optimization steps = 60


Training:   0%|          | 0/60 [00:00<?, ?it/s]

In [ ]:
# = = = = = = = = = = = = = = = = Train = = = = = = = = = = = = = = = = = = =
total_batch_size = args.per_device_train_batch_size * \
    accelerator.num_processes * args.gradient_accumulation_steps

logger.info("***** Running training *****")
logger.info(f" Num examples = {len(train_dataset)}")
logger.info(f" Num Epochs = {args.num_train_epochs}")
logger.info(
    f" Instantaneous batch size per device = {args.per_device_train_batch_size}")
logger.info(
    f" Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(
    f" Gradient Accumulation steps = {args.gradient_accumulation_steps}")
logger.info(f" Total optimization steps = {args.max_train_steps}")

# Only show the progress bar once on each machine.
progress_bar = tqdm(range(args.max_train_steps), desc="Training: ",
                    disable=not accelerator.is_local_main_process)
completed_steps = 0

val_results = []
acc_losses = []
best_r2_f1 = None
best_epoch = 0

# edit #
if args.model_type == 'bart' or args.model_type == 't5':
    # task_specific_params = model.module.config.task_specific_params
    task_specific_params = model.config.task_specific_params
    params = task_specific_params.get('summarization', {})
    params['min_length'] = args.min_target_length
    params['max_length'] = args.max_target_length
    params['length_penalty'] = args.length_penalty
    params['num_beams'] = args.num_beams
    # model.module.config.update(params)
    model.config.update(params)
else:
    raise ValueError(
        '{} model type not implemented'.format(args.model_type))

loss_list = []
train_loss_list = []
val_loss_list = []
last_output = None
hidden_states = None

# =  =  =  =  =  =  =  =  =  =  =  =  =  =  =  = Train =  =  =  =  =  =  =  =  =  =  =  =  =  =  =
for epoch in range(args.num_train_epochs):
    # train
    model.train()
    epoch_loss = 0.0
    for step, batch in enumerate(train_dataloader):

        if args.ctrlen_model:  # CTRLen model
            outputs, loss = model(batch, tokenizer)
        # w/ and w/o label smoothing (always better with label smoothing)
        else:
            if args.label_smoothing == 0:
                outputs = model(**batch)
                loss = outputs.loss
            else:
                outputs = model(**batch, output_hidden_states=True)
                last_output = outputs
                output_logits = outputs.logits
                hidden_states = outputs.decoder_hidden_states
                # print(f"logits: {output_logits.shape}")
                # print("="*100)
                # # print(f"hidden states: {hidden_states.shape}")
                # print(f"loss: {outputs.loss}")
                # print(outputs.keys())
                # print("="*100)
                output_probs = torch.nn.functional.log_softmax(
                    output_logits, dim=-1)
                # edit #
                # output_probs = output_probs.view(-1,
                #                                  model.module.config.vocab_size)
                output_probs = output_probs.view(-1,
                                                 model.config.vocab_size)

                gt_logits = batch['labels']
                # print(f"label: {gt_logits.shape}")
                # print("="*100)
                gt_logits = gt_logits.view(-1)

                # print(f"output_probs: {output_probs.shape}")
                # print("-"*100)
                # print(f"gt_logits: {gt_logits.shape}")
                # print("="*100)

                loss_nll, nll = label_smoothed_nll_loss(
                    output_probs, gt_logits, args.label_smoothing, ignore_index=tokenizer.pad_token_id)

                cosine_loss = torch.nn.CosineEmbeddingLoss()

                loss_cs = cosine_loss(outputs.encoder_last_hidden_state[0], outputs.encoder_last_hidden_state[1], torch.ones(outputs.encoder_last_hidden_state.size(dim=1)).to(torch.device('cuda')))

                alpha = 0.5

                loss = loss_nll + alpha * (1 - loss_cs)

                # print(f"loss_fn: {loss}")
                # print("-"*100)
                # print(f"nll: {nll}")
                # print("="*100)

        acc_losses.append(loss.item())
        loss_list.append(loss)
        epoch_loss += loss.item()
        loss = loss / args.gradient_accumulation_steps
        # print(f"loss_grad: {loss}")
        accelerator.backward(loss)
        # break

        if step % args.gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)
            progress_bar.set_postfix(lr=lr_scheduler.get_last_lr()[
                                     0], loss=np.mean(acc_losses[-50:]))
            completed_steps += 1
            train_loss_list.append(epoch_loss/len(batch))

        if completed_steps >= args.max_train_steps:
            break

    # # =  =  =  =  =  =  =  =  =  =  =  =  =  =  =  = EVAL =  =  =  =  =  =  =  =  =  =  =  =  =  =  =
    model.eval()
    val_predict = []
    val_groundtruth = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            val_loss = []
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"]
            )

            # print(generated_tokens)
            # print("="*100)

            generated_tokens = accelerator.pad_across_processes(
                generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
            )

            # print(generated_tokens)
            # print("="*100)

            labels = batch["labels"]
            if not args.pad_to_max_length:
                # If we did not pad to max length, we need to pad the labels too
                labels = accelerator.pad_across_processes(
                    batch["labels"], dim=1, pad_index=tokenizer.pad_token_id)

            # print(generated_tokens)
            # print("="*100)
            # print(labels)
            # print("="*100)
            # loss, _ = label_smoothed_nll_loss_gen(generated_tokens, labels, args.label_smoothing, ignore_index=tokenizer.pad_token_id)
            # break
            # loss, _ = map(label_smoothed_nll_loss, generated_tokens, labels, args.label_smoothing, ignore_index=tokenizer.pad_token_id)
            # val_loss.extend(loss)

            generated_tokens = accelerator.gather(
                generated_tokens).cpu().numpy()
            labels = accelerator.gather(labels).cpu().numpy()

            if args.ignore_pad_token_for_loss:
                # Replace -100 in the labels as we can't decode them.
                labels = np.where(labels != -100, labels,
                                tokenizer.pad_token_id)
            if isinstance(generated_tokens, tuple):
                generated_tokens = generated_tokens[0]

            # print(generated_tokens[0])
            # print("="*100)
            # print(labels[0])
            # loss, _ = label_smoothed_nll_loss(generated_tokens[0], labels[0], args.label_smoothing, ignore_index=tokenizer.pad_token_id)
            # break

            decoded_preds = tokenizer.batch_decode(
                generated_tokens, skip_special_tokens=True)
            decoded_labels = tokenizer.batch_decode(
                labels, skip_special_tokens=True)

            # print(decoded_preds)
            # print("="*100)
            # print(decoded_labels)
            # print("="*100)

            decoded_preds, decoded_labels = postprocess_text(
                decoded_preds, decoded_labels)

            # print(decoded_preds)
            # print("="*100)
            # print(decoded_labels)
            # print("="*100)
            # if step == 1:
            #     break

            val_predict.extend(decoded_preds)
            val_groundtruth.extend(decoded_labels)

    if args.len_output == 'real':
        new_val_predict = []
        for sample in val_predict:
            try:
                gen_sum = sample.split('Summary: ')[2]
                new_val_predict.append(gen_sum)
            except:
                new_val_predict.append(sample)
        val_predict = new_val_predict
    else:
        new_val_predict = val_predict

    logger.info("")
    logger.info("Rouge score on val set after epoch {}".format(epoch+1))
    eval_results = py_rouge_scores(val_predict, val_groundtruth)

    if best_r2_f1 is None:
        best_r2_f1 = eval_results
    if eval_results['rouge-2']['f'] >= best_r2_f1['rouge-2']['f']:
        best_r2_f1 = eval_results
        best_epoch = epoch + 1

        os.makedirs(args.output_dir+'/best', exist_ok=True)
        accelerator.wait_for_everyone()
        unwrapped_model = accelerator.unwrap_model(model)
        unwrapped_model.save_pretrained(
            args.output_dir+'/best', save_function=accelerator.save)
        if accelerator.is_main_process:
            tokenizer.save_pretrained(args.output_dir+'/best')
        # save vocab
        vocab = tokenizer.vocab.copy()
        vocab = {k: v for k, v in sorted(
            vocab.items(), key=lambda item: item[1])}
        with open(args.output_dir + '/best/vocab.txt', 'w') as f:
            for word, index in vocab.items():
                # it lead to encoding bug on some machines, so i add this line
                word = word.encode('ascii', 'ignore').decode('ascii')
                f.write(str(index) + ': ' + word + '\n')

    # = = = = = = = = = = = = = = = = = = = = = = = = =
    logger.info(
        "Current Best Validation Result is at epoch {}".format(best_epoch))
    py_rouge_scores(None, None, best_r2_f1)


10/04/2023 18:57:09 - INFO - __main__ - ***** Running training *****
10/04/2023 18:57:09 - INFO - __main__ -  Num examples = 1500
10/04/2023 18:57:09 - INFO - __main__ -  Num Epochs = 5
10/04/2023 18:57:09 - INFO - __main__ -  Instantaneous batch size per device = 2
10/04/2023 18:57:09 - INFO - __main__ -  Total train batch size (w. parallel, distributed & accumulation) = 128
10/04/2023 18:57:09 - INFO - __main__ -  Gradient Accumulation steps = 64
10/04/2023 18:57:09 - INFO - __main__ -  Total optimization steps = 60


Training:   0%|          | 0/60 [00:00<?, ?it/s]

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "max_length": 128,
  "min_length": 1,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.33.3"
}

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Gene

In [ ]:
len(outputs.encoder_last_hidden_state)

2

In [ ]:
outputs.encoder_last_hidden_state

tensor([[[ 0.0199,  0.0383,  0.0413,  ..., -0.0192,  0.0033, -0.0034],
         [-0.1682, -0.0113,  0.1301,  ..., -0.0883, -0.0093, -0.1615],
         [-0.0111, -0.4472,  0.0442,  ...,  0.0226, -0.0170, -0.2872],
         ...,
         [-0.0308, -0.2955, -0.3913,  ...,  0.0104, -0.0009, -0.0810],
         [-0.0451, -0.4547, -0.5451,  ...,  0.1203, -0.0126,  0.1627],
         [-0.0310, -0.0535, -0.3161,  ...,  0.0269, -0.0641,  0.1126]],

        [[ 0.0211,  0.0319,  0.0484,  ..., -0.0260, -0.0024, -0.0104],
         [-0.2227, -0.0779, -0.0718,  ...,  0.0067, -0.0177, -0.0376],
         [-0.1187, -0.5692, -0.0248,  ..., -0.0780,  0.0284, -0.4738],
         ...,
         [-0.0440,  0.1978, -0.3247,  ..., -0.0017,  0.1312,  0.3230],
         [-0.1271, -0.3494, -0.5145,  ...,  0.0051,  0.1007,  0.0965],
         [-0.1246, -0.2684, -0.6091,  ...,  0.0990,  0.0489, -0.0175]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward0>)

In [ ]:
outputs.encoder_last_hidden_state.size(dim=1)

296

In [ ]:
print(len(outputs.logits))
print(outputs.logits[0].size())

2
torch.Size([48, 50266])


In [ ]:
cosine_loss = torch.nn.CosineEmbeddingLoss()

In [ ]:
loss_cs = cosine_loss(outputs.encoder_last_hidden_state[0], outputs.encoder_last_hidden_state[1], torch.ones(outputs.encoder_last_hidden_state.size(dim=1)).to(torch.device('cuda')))

In [ ]:
loss_cs

tensor(0.7458, device='cuda:0', grad_fn=<MeanBackward0>)

In [ ]:
# =  =  =  =  =  =  =  =  =  =  =  =  =  =  =  = Test =  =  =  =  =  =  =  =  =  =  =  =  =  =  =  =  =  =  =
# load best model
logger.info("Loading Best Result is at epoch {} for Testing".format(best_epoch))

unwrapped_model = accelerator.unwrap_model(model)
config          = config.from_pretrained(args.output_dir+'/best')
tokenizer       = tokenizer.from_pretrained(args.output_dir+'/best', config=config)
unwrapped_model = unwrapped_model.from_pretrained(args.output_dir+'/best', config=config)
model           = accelerator.prepare(unwrapped_model)

if args.model_type == 'bart' or args.model_type == 't5':
    task_specific_params = model.config.task_specific_params
    params = task_specific_params.get('summarization', {})
    params['min_length'] = args.min_target_length
    params['max_length'] = args.max_target_length
    params['length_penalty'] = args.length_penalty
    params['num_beams'] = args.num_beams
    model.config.update(params)
else:
    raise ValueError('{} model type not implemented'.format(args.model_type))

# start Test
logger.info("Collecting Testing Result...")
model.eval()

test_predict     = []
test_groundtruth = []
for step, batch in enumerate(tqdm(test_dataloader, leave=False)):
    with torch.no_grad():
        generated_tokens = accelerator.unwrap_model(model).generate(
            batch["input_ids"],
            attention_mask=batch["attention_mask"],
        )

        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = batch["labels"]

        if not args.pad_to_max_length:
            # If we did not pad to max length, we need to pad the labels too
            labels = accelerator.pad_across_processes(batch["labels"], dim=1, pad_index=tokenizer.pad_token_id)

        generated_tokens = accelerator.gather(generated_tokens).cpu().numpy()
        labels = accelerator.gather(labels).cpu().numpy()

        if args.ignore_pad_token_for_loss:
            # Replace -100 in the labels as we can't decode them.
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        if isinstance(generated_tokens, tuple):
            generated_tokens = generated_tokens[0]

        decoded_preds  = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

        decoded_preds  = [' '.join(sent.split('\n')) for sent in decoded_preds]
        decoded_labels = [' '.join(sent.split('\n')) for sent in decoded_labels]

        test_predict.extend(decoded_preds)
        test_groundtruth.extend(decoded_labels)

print(raw_datasets['test']['dialogue'][0])

if args.len_output == 'real':
    new_test_predict = []
    for sample in test_predict:
        try:
            gen_sum = sample.split('Summary: ')[2]
            new_test_predict.append(gen_sum)
        except:
            new_test_predict.append(sample)
    test_predict = new_test_predict

logger.info("")
logger.info("ROUGE score on test set")
test_scores = py_rouge_scores(test_predict, test_groundtruth)
logger.info("")


# Save generated summaries
if args.len_input == 'predict':
    os.makedirs(args.output_dir+'/predict_gen_samples', exist_ok=True)
else:
    os.makedirs(args.output_dir+'/gen_samples', exist_ok=True)

for i in range(len(test_predict)):
    test_id        = raw_datasets['test']['id'][i]
    test_dialogue  = raw_datasets['test']['dialogue'][i]
    test_summary   = raw_datasets['test']['summary'][i]
    test_predict_s = test_predict[i]

    if args.len_input == 'predict':
        with open(args.output_dir+'/predict_gen_samples/'+str(test_id)+'.txt', 'w') as f:
            test_dialogue = test_dialogue.encode('ascii', 'ignore').decode('ascii')
            f.write(test_dialogue)
            f.write('\n\n')
            f.write('Golden Summary:\n')
            test_summary = test_summary.encode('ascii', 'ignore').decode('ascii')
            f.write(test_summary)
            f.write('\n\n')
            f.write('Generate Summary:\n')
            test_predict_s = test_predict_s.encode('ascii', 'ignore').decode('ascii')
            f.write(test_predict_s)
    else:
        with open(args.output_dir+'/gen_samples/'+str(test_id)+'.txt', 'w') as f:
            test_dialogue = test_dialogue.encode('ascii', 'ignore').decode('ascii')
            f.write(test_dialogue)
            f.write('\n\n')
            f.write('Golden Summary:\n')
            test_summary = test_summary.encode('ascii', 'ignore').decode('ascii')
            f.write(test_summary)
            f.write('\n\n')
            f.write('Generate Summary:\n')
            test_predict_s = test_predict_s.encode('ascii', 'ignore').decode('ascii')
            f.write(test_predict_s)

10/04/2023 19:07:23 - INFO - __main__ - Loading Best Result is at epoch 4 for Testing
loading configuration file ./output/1-bart-baseline-loss/best/config.json
Model config BartConfig {
  "_name_or_path": "facebook/bart-large",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"

  0%|          | 0/19 [00:00<?, ?it/s]

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "max_length": 128,
  "min_length": 1,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.33.3"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "max_length": 128,
  "min_length": 1,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.33.3"
}

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "max_length": 128,
  "min_length": 1,
  "no_re

Topic of Summary: communication method. Length of Summary: 27. Dialogue: # Person1 # : Ms. Dawson , I need you to take a dictation for me . # Person2 # : Yes , sir ... # Person1 # : This should go out as an intra-office memorandum to all employees by this afternoon . Are you ready ? # Person2 # : Yes , sir . Go ahead . # Person1 # : Attention all staff ... Effective immediately , all office [TAG]communications[TAG] are restricted to email correspondence and official memos . The use of Instant Message programs by employees during working hours is strictly prohibited . # Person2 # : Sir , does this apply to intra-office [TAG]communications[TAG] only ? Or will it also restrict external [TAG]communications[TAG] ? # Person1 # : It should apply to all [TAG]communications[TAG] , not only in this office between employees , but also any outside [TAG]communications[TAG] . # Person2 # : But sir , many employees use Instant Messaging to communicate with their clients . # Person1 # : They will just

10/04/2023 19:08:05 - INFO - root - 
10/04/2023 19:08:05 - INFO - root - 	rouge-1:	P: 37.12	R: 47.88	F1: 40.73
10/04/2023 19:08:05 - INFO - root - 	rouge-2:	P: 13.30	R: 17.25	F1: 14.64
10/04/2023 19:08:05 - INFO - root - 	rouge-l:	P: 35.01	R: 43.23	F1: 37.96
10/04/2023 19:08:05 - INFO - root - 
10/04/2023 19:08:05 - INFO - __main__ - 
